In [0]:
using Pkg
Pkg.add("CuArrays")
Pkg.add("CUDAnative")
Pkg.add("CUDAdrv")
Pkg.add("BenchmarkTools")
Pkg.add("Test")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [0]:
using CUDAnative, CUDAdrv, CuArrays, BenchmarkTools, Test

Internal error: encountered unexpected error in runtime:
BoundsError(a=Array{Core.Compiler.BasicBlock, (32,)}[
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=1, last=7), preds=Array{Int64, (1,)}[32], succs=Array{Int64, (1,)}[2]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=8, last=14), preds=Array{Int64, (1,)}[1], succs=Array{Int64, (2,)}[5, 3]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=15, last=20), preds=Array{Int64, (1,)}[2], succs=Array{Int64, (1,)}[4]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=21, last=21), preds=Array{Int64, (1,)}[3], succs=Array{Int64, (1,)}[7]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=22, last=28), preds=Array{Int64, (1,)}[2], succs=Array{Int64, (1,)}[6]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=29, last=29), preds=Array{Int64, (1,)}[5], succs=Array{Int64, (1,)}[7]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=30, last=

In [0]:
function parallel_red!(a,temp)
  idx=threadIdx().x
  if idx <= min(blockDim().x,size(a,1))
  step=1
  
  chunk=ceil(Int,size(a,1)/blockDim().x)
  
  while step < size(a,1)   #size(a,1)
  
    for i=1:chunk
      k = i + (idx-1)*chunk
      if(k<=size(a,1))
        @inbounds temp[k]=a[k]
      end
    end
   
   sync_threads()
   
    for i=1:chunk
      k = i + (idx-1)*chunk
        if(k<=size(a,1) && (k % size(a,1) > step || k % size(a,1)==0))
          temp[k]=a[k]+a[k-step]    
        end
    end
    
  sync_threads()
     
   for i=1:chunk
      k = i + (idx-1)*chunk
      if(k<=size(a,1))
        @inbounds a[k]=temp[k]
      end
   end
  
  sync_threads()
  
  
  step*=2
  end  #while loop
  
    
end  #upper if
return nothing
end

parallel_red! (generic function with 1 method)

In [0]:
n=1000000
t=500 # t is number of threads to be launched HERE IF I KEEP t=500 THEN IT IS FINE BUT IF INCREASE THEN THAT SAME ERROR- TOO MANY RESOURCES REQUESTED FOR LAUNCH
h_a=rand(n)
h_temp=zeros(t*ceil(Int,size(h_a,1)/t))

1000000-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [0]:
d_a=CuArray(h_a)
d_temp=CuArray(h_temp)
@cuda threads=t parallel_red!(d_a,d_temp)

In [0]:
h_a1=Array(d_a)
h_a1

1000000-element Array{Float64,1}:
      0.5852986923815144
      1.1517334544663003
      1.3371990086166887
      1.914367907290707 
      2.3479222687596453
      2.4695013700801365
      2.9171237983159113
      3.2674334658474207
      3.550139729967963 
      4.001861869307547 
      4.726531133102291 
      5.480873517901575 
      6.445129345473139 
      ⋮                 
 499918.51683153154     
 499919.2482709616      
 499919.54023765953     
 499919.57160999975     
 499920.11495942704     
 499920.1916015734      
 499920.8207648557      
 499921.068018122       
 499921.7116014669      
 499922.3451446641      
 499922.931177265       
 499923.01404396485     

In [0]:
function seq_red!(a)
  for i=2:size(a,1)
    a[i]+=a[i-1]
  end
  return a
end

seq_red! (generic function with 1 method)

In [0]:
sequential_array=seq_red!(h_a)

1000000-element Array{Float64,1}:
      0.5852986923815144
      1.1517334544663003
      1.3371990086166887
      1.914367907290707 
      2.3479222687596453
      2.4695013700801365
      2.9171237983159117
      3.2674334658474207
      3.550139729967963 
      4.001861869307547 
      4.726531133102291 
      5.480873517901575 
      6.4451293454731395
      ⋮                 
 499918.5168315329      
 499919.2482709629      
 499919.54023766087     
 499919.5716100011      
 499920.1149594283      
 499920.1916015747      
 499920.820764857       
 499921.0680181232      
 499921.71160146815     
 499922.34514466545     
 499922.9311772663      
 499923.0140439662      

In [0]:
@test isapprox(sequential_array,h_a1)

Test Passed

In [0]:
@benchmark sequential_array=seq_red!(h_a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.861 ms (0.00% GC)
  median time:      2.896 ms (0.00% GC)
  mean time:        2.944 ms (0.00% GC)
  maximum time:     3.846 ms (0.00% GC)
  --------------
  samples:          1695
  evals/sample:     1

In [0]:
@benchmark @cuda threads=t parallel_red!(d_a,d_temp)

BenchmarkTools.Trial: 
  memory estimate:  1.70 KiB
  allocs estimate:  52
  --------------
  minimum time:     20.357 μs (0.00% GC)
  median time:      156.707 ms (0.00% GC)
  mean time:        143.513 ms (0.00% GC)
  maximum time:     156.803 ms (0.00% GC)
  --------------
  samples:          35
  evals/sample:     1